In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import cv2
import cvlib as cv

# Load the model directly (assuming it's in a compatible format for your TensorFlow version)
model = tf.keras.models.load_model('path_to_saved_model_folder')  # Replace with the correct path

# Open webcam
webcam = cv2.VideoCapture(0)
classes = ['man', 'woman']

# Loop through frames
while webcam.isOpened():
    # Read frame from webcam
    status, frame = webcam.read()
    
    # Apply face detection
    faces, confidence = cv.detect_face(frame)

    # Loop through detected faces
    for idx, f in enumerate(faces):
        # Get corner points of face rectangle
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # Draw rectangle over face
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # Crop the detected face region
        face_crop = np.copy(frame[startY:endY, startX:endX])

        # Ignore if face is too small
        if face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
            continue

        # Preprocess for gender detection model
        face_crop = cv2.resize(face_crop, (96, 96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # Predict gender
        conf = model.predict(face_crop)[0]
        idx = np.argmax(conf)
        label = classes[idx]
        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        # Display label and confidence
        Y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.putText(frame, label, (startX, Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Display output
    cv2.imshow("Gender Detection", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
webcam.release()


ModuleNotFoundError: No module named 'cvlib'